# Convert Keras model to TFlight 

Here we want to convert a keras model (keras_vggface) into a Tensorflow-light model to be able to use it for faster inference on an embedded system. Also we want to be able to do post-training-quantisation. This is also done via the Tensorflow-light module.

Please install keras_vggface beforehand with: 

pip install git+https://github.com/rcmalli/keras-vggface.git

In [7]:
import cv2
from pathlib import Path
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.version.VERSION)
tf.keras.backend.clear_session()  # For easy reset of notebook state.


2.1.0


 From: https://github.com/tensorflow/tensorflow/issues/31057

**Explanation:** 

 As you didn't run the your model, it does not know the shape and dtype of the input data it should be expecting, and thus cannot create its weight variables.
 
 To be eiter saved as pb or converted to tensorflowLight, the model needs to be initiatetd, either with:
 fit, or run , etc.

**Workaround:**

https://github.com/tensorflow/tensorflow/issues/31057#issuecomment-523262141 
 One workaround is that after finished custom training, call model.predict on some input tensors and then call model.save to save your model as SavedModel format.
 
 
 **also:**
 
 https://github.com/tensorflow/tensorflow/issues/31057#issuecomment-560335565

 As @jvishnuvardhan pointed, using _set_inputs does the work. However, I think it should not be necessary if the model is called and fed an input. In other words, the call function of Model should automatically call _set_inputs with the input received

**again:**

https://github.com/tensorflow/tensorflow/issues/31057#issuecomment-580192402

 if you use model.predict(...) instead of model(...) it works.
 
**might not work:**

 https://github.com/tensorflow/tensorflow/issues/31057#issuecomment-589172098
 I can't save a custom model even though I successfully call model.predict(...) before the save. I get all kinds of error. My model is a seq2seq model. Why is tensorflow saying the subclassing keras.Model is the recommended low-level/flexible way when i is almost impossible to save it as a SavedModel?...

## load image for single run

In [8]:
folder_idx='Aaron Eckhart'
image_idx='000001.jpg'

pfad=str(Path.cwd() / 'sizeceleb_224_224' / str(folder_idx) / str(image_idx))

img=cv2.imread(pfad) 

if img.shape > (224,224): #downsampling
    img=cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
pixels = img.astype('float32')
sample = np.expand_dims(pixels, axis=0)
sample = preprocess_input(sample, version=2)


Run the model with a the single image to set the input

In [ ]:
resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
                                pooling='avg')  # pooling: None, avg or max


resnet50_features.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

!mkdir -p saved_model        

def case(Fall):
    print("Fall: ", Fall)
    if Fall==1:
        # VERSION 1 Run/Transfrom Keras model into tf model, then save as h5
        EMBEDDINGS = resnet50_features.predict(sample)
        tf_model = tf.keras.models.Model(resnet50_features)
        EMBEDDINGS = tf_model.predict(sample)
        resnet50_features.save('saved_model/my_model.h5')
        
    if Fall==2:
        # VERSION 2 Only run the Keras model, then save as pb
        EMBEDDINGS = resnet50_features.predict(sample)
        tf.saved_model.save(resnet50_features, 'saved_model/my_model2.pb')
        
    if Fall==3:
        #VERSION 3 Run/Transform Keras into tf model, then save as pb
        EMBEDDINGS = resnet50_features.predict(sample)
        tf_model = tf.keras.models.Model(resnet50_features)
        EMBEDDINGS = tf_model.predict(sample)
        tf.saved_model.save(resnet50_features, 'saved_model/my_model.pb')
        
    if Fall==4:
        #VERSION 4 Run/Transform Keras model into tf model and convert to tfligth model
        EMBEDDINGS = resnet50_features.predict(sample)
        tf_model = tf.keras.models.Model(resnet50_features)
        EMBEDDINGS = tf_model.predict(sample)
        
        converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
        tflite_model = converter.convert()
        
    if Fall==5:
        # VERSION 5 Only run the Keras model, then save as h5
        #EMBEDDINGS = resnet50_features.predict(sample)    
        resnet50_features.save('saved_model/my_model.h5') 
        
    if Fall==6:
        # Version 6 cast the Keras model into a TF model (https://github.com/keras-team/keras/issues/9310#issuecomment-391350641)
        myKerasModel = build_keras_model(resnet50_features) # put your keras model here
        tfCompatibleMod = tf.keras.models.Model(myKerasModel) # cast to type tensorflow expect
        tfCompatibleMod.compile(...) # compile
        estimator = tf.keras.estimator.model_to_estimator(keras_model=tfCompatibleMod)
case(5)


Fall 5 will work but does not really solve the issue. But lets try to convert the h5 into pb. 

Call the keras to pb tool from: https://github.com/amir-abdi/keras_to_tensorflow
to transpose the h5 saved model to a pb file. It does nit run under Tensorflow 2.0 so use a enviroment with e.g., TF 1.15.

If the subporcess does snot work, just use the terminal and call:

python3 h5_to_pb/keras_to_tensorflow.py --input_model="/saved_model/my_model.h5" --output_model="/saved_model/my_model3.pb"

In [ ]:
import os
from pathlib import Path
import subprocess

# New process, connected to the Python interpreter through pipes:
subprocess.run(['bash','-c','python3 h5_to_pb/keras_to_tensorflow.py --input_model="/saved_model/my_model.h5" --output_model="/saved_model/my_model3.pb"'], capture_output=True)
#prog = subprocess.Popen(['bash','-c','python3 h5_to_pb/keras_to_tensorflow.py --input_model="/saved_model/my_model.h5" --output_model="/saved_model/my_model3.pb"'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#prog.communicate()  # Returns (stdoutdata, stderrdata): stdout and stderr are ignored, here
#if prog.returncode:
 #   raise Exception('program returned error code {0}'.format(prog.returncode))


In [4]:
#Seems to work if the model is manually compiled


import tensorflow as tf

model = tf.keras.models.load_model('saved_model/my_model.h5')
model.compile(optimizer='sgd', loss='mean_squared_error')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("saved_model/converted_model.tflite", "wb").write(tflite_model)

# # #converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# # converter = tf.lite.TFLiteConverter.from_keras_model(resnet50_features)
# # converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# # tflite_quant_model = converter.convert()

abbhbz


93951728

In [ ]:
import logging
import tensorflow as tf
from tensorflow.compat.v1 import graph_util
from tensorflow.python.keras import backend as K
from tensorflow import keras

# necessary !!!
tf.compat.v1.disable_eager_execution()

h5_path = 'saved_model/my_model.h5'
model = keras.models.load_model(h5_path)
model.summary()
# save pb
with K.get_session() as sess:
    output_names = [out.op.name for out in model.outputs]
    input_graph_def = sess.graph.as_graph_def()
    for node in input_graph_def.node:
        node.device = ""
    graph = graph_util.remove_training_nodes(input_graph_def)
    graph_frozen = graph_util.convert_variables_to_constants(sess, graph, output_names)
    tf.io.write_graph(graph_frozen, 'saved_model', 'model.pb', as_text=False)
logging.info("save pb successfully！")

In [ ]:
import tensorflow as tf

saved_model_dir='saved_model'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()


In [ ]:
model.save('./my_TFl_model') 